# Exercise: Convolutional Neural Networks

In this exercise session, we will learn about PyTorch, one of the most used framework for deep learning in python. We will use it to implement our own neural networks and train them through gradient descent.


We will use the following packages:

`torch`: The framework we will use for training deep nets, with useful sub-modules `torch.nn` and `torch.nn.functional` that we import below.

`torchvision`: Helper package consisting of popular datasets, model architectures, and common image transformations for computer vision. We will use it for loading the MNIST dataset and to perform simple data transformations.

`torchinfo`: Helper package for visualizing deep net architectures.

In [1]:
# 3rd party
import numpy as np

# We import PyTorch and some of its internal modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Lambda, Compose
from torchinfo import summary
import matplotlib.pyplot as plt
from medmnist import DermaMNIST

# Project files.
from helpers import accuracy, DrawingPad

In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

# 1 PyTorch

## 1.1 Motivation

In the first part of the exercise we will revisit the MNIST dataset of hand-written digits, and we will train deep net models to classify the digits. Instead of doing all the hard coding work manually, we will simplify our life by using a deep learning framework: **PyTorch**.

Last week we have implemented our own Multi-Layer Perceptron (MLP), where we defined both the forward pass and back-propagation together with a simple optimizer (stochastic gradient descent, SGD, update rule) and successfully trained it to perform classification. Given the amount of code written, one can imagine that prototyping with various NN architectures and training strategies might get tedious. That is where PyTorch (and other deep learning frameworks) come into play.

## 1.2 About PyTorch

[PyTorch](https://pytorch.org/) is an optimized tensor library for deep learning using GPUs and CPUs. It allows
for fast prototyping by providing high-level access to all necessary building blocks, including NN layers, activation functions, loss functions, and optimizers, to name a few. Most importantly, however, PyTorch implements the [autograd](https://pytorch.org/docs/stable/autograd.html) package, which allows for automatic differentiation of the operations we use to define NN architectures. In other words, one only has to implement the forward pass, namely to combine desired layers, while the **backpropagation is computed automatically**.

## 1.3 Basic pipeline

To define and train deep net models, one would usually implement the following steps:

    1. Load the dataset.
    2. Define and instantiate a deep net architecture.
    3. Choose or implement a loss function (such as the mean squared error).
    4. Choose and instantiate an optimizer (such as the SGD).
    5. Repeating multiple time for the whole dataset, for each batch in the dataset:
        5.1. Load a batch.
        5.2. Run a forward pass through your model.
        5.3. Compute the loss.
        5.4. Run a backward pass, i.e., compute gradients of the loss w.r.t. the trainable parameters (weights).
        5.5. Update the weights using the optimizer.
        5.6. Zero-out the accumulated gradients before the next iteration.
        
We will see this exact pipeline in our code as well.

## 1.4 Essential bulding blocks

This section gives a high-level summary of the most important components representing the bare minimum that you will need to start playing with PyTorch and deep net models. You might want to skim through the official tutorials as well, namely [What is PyTorch](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) and [Neural Networks](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py). Here is the list of the components that will be explained in more detail along with the code blocks.

  - **nn.Module**: Base class for NN architectures.
  - **criterion**: A loss function.
  - **backward-pass**: Derivatives computed by the auto-differentiation system.
  - **optimizer**: Updates the trainable parameters (weights) during training.

## 1.5 Loading the data

We are at step (1) of the training pipeline where we prepare the data. In PyTorch, loading the data is traditionally performed by creating:
* a *dataset* that manages the loading and transformations of the data
* and a *dataloader* that is a Python *iterator*, which returns the batches of data and associated labels from our dataset.

PyTorch provides us with the `Dataset` and `DataLoader` classes for this.

As was the case of previous week, we will work with the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset, where each sample is stored as a $28 \times 28$ pixels grayscale image. The data samples are loaded as `torch.Tensor` data type, multi-dimentional matrices similar to `numpy.ndarray`.

`MNIST` below is a sub-class of `Dataset`, which will download the dataset when used for the first time. It returns an image $x$ and its true label $y$. The dataloader will then prepare batches out of these images and labels.

In [3]:
batch_size = 128

# Dataset and DataLoader for MLP.
#dataset_train = MNIST('data', train=True, download=True, transform=ToTensor())
#dataset_test = MNIST('data', train=False, download=True, transform=ToTensor())
#dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
#dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

train_dataset = DermaMNIST(split="train", download=download, size=28)
test_dataset = DermaMNIST(split="test", download=download, size=28)

train_images = np.stack([np.array(image) for image, _ in train_dataset])
test_images = np.stack([np.array(image) for image, _ in test_dataset])

train_labels = np.array([label for _, label in train_dataset]).reshape(-1)
test_labels = np.array([label for _, label in test_dataset]).reshape(-1)

dataset_train = TensorDataset(torch.from_numpy(train_images).float(),
                                      torch.from_numpy(training_labels))
test_dataset = TensorDataset(torch.from_numpy(training_data).float(),
                                      torch.from_numpy(training_labels))
train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

print('Loaded {} train and {} valid samples.'.format(len(dataset_train), len(dataset_test)))

Using downloaded and verified file: /home/poca/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/poca/.medmnist/dermamnist.npz
Loaded 7007 train and 2005 valid samples.


## 1.6 Multi-Layer Perceptron (MLP)

### Architecture

We are at step (2) of the training pipeline. We will start by implementing an MLP consisting of a 1D input layer (we flatten, i.e., vectorize, the input image) of shape ($784$, ), $3$ hidden fully connected layers and an output layer of shape ($10$, ), as we have $10$ classes.

As you saw last week, one layer of an MLP computes the following function:

$$ \mathbf{y} = \sigma \left(\mathbf{W}^\top\mathbf{x} + \mathbf{b}\right), $$

where $\sigma$ is the activation, $\mathbf{W}$ is the weight, and $\mathbf{b}$ the bias. The type of layer that computes $\mathbf{W}^\top\mathbf{x} + \mathbf{b}$ is referred to as *fully-connected* (FC) because every input is connected to every output (recall the MLP diagram from the lectures). Additionally, we also call them *linear* layers because they compute a linear function with respect to their input (plus a bias).

### Optimization criterion

We would like to interpret the output vector $\hat{\mathbf{y}} \in \mathbb{R}^{10}$ as the probabilities of data sample $\mathbf{x} \in \mathbb{R}^{784}$ belonging to each class $j \in \{0, 1, 2, ..., 9\}$. Therefore, we will make use of the activation function **softmax**, defined as

$$ \hat{y}_j = P(\text{class}=j|\mathbf{z}) = \mathrm{Softmax}_j(\mathbf{z}) = \frac{\exp{z_j}}{\sum_{k=0}^{9}{\exp{z_k}}}, $$

on the final output of our network $\mathbf{z}$ (these values pre-softmax are referred to as "logits").
The softmax guarantees that $\sum_{j=0}^{9}\hat{y}_{j} = 1$ and $\hat{y}_j \geq 0, \, \forall j$, meaning that the predicted vector $\hat{\mathbf{y}}$ is indeed a valid probability distribution over classes.

Finally, we would like to match the predicted distribution $\hat{\mathbf{y}}$ to the ground truth (GT) one $\mathbf{y}$, where $\mathbf{y}$ is given as a one-hot encoding ($\mathbf{y}$ is all zeros except for a $1$ at the index $j$, if $j$ is the correct class to be predicted). The optimization criterion of choice is then to minimize the [**cross-entropy**](https://en.wikipedia.org/wiki/Cross_entropy) (CE) of $\hat{\mathbf{y}}$ and $\mathbf{y}$. Therefore our final loss function $\mathcal{L}$ is defined as:

$$ \mathcal{L} = \text{CE}\left(\hat{\mathbf{y}}, \mathbf{y}\right).$$

Thankfully, PyTorch provides the implementation of $\mathcal{L}$ that directly works with the logits, **so you will only really need to provide the output $\mathbf{z}$** (i.e. the 10-dimensional output of your last layer *before* the softmax). We will get back to $\mathcal{L}$ later.

---

### nn.Module
Each custom NN architecture you choose to implement has to subclass the [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=nn+module#torch.nn.Module), which conveniently keeps track of all the trainable parameters. From the programmer's perspective, you have to implement the constructor (`__init__`) and override the `forward()` function:

- **`__init__()`**

You will define your layers (e.g., fully connected layer, 2D convolutional layer, etc.) in the constructor, and `nn.Module` will automatically keep track of all the weights these layers contain. Here, we basically prepare and initialized what we need in the network.

- **`forward()`**

This function really defines the architecture, as you will sequentally call your layers in the desired order. Each time you call `forward()` (every training iteration), the so-called **computational graph** is built. It is a directed acyclic graph (DAG) of nodes corresponding to the operations you have called. Each node defines the derivative of its outputs w.r.t. its inputs. The computational graph is then traversed in the reversed fashion once you call `backward()` and the derivatives are computed.

**Note:** PyTorch allows us to use call our model like a function on the input data like `y = model(x)`. This will automatically call the `forward()` function of the model.

All the trainable parameters, which your model consists of, can be accessed via a call to `model.parameters()` implemented in `nn.Module`. This comes in handy once instantiating your optimizer as you have to pass all the parameters you want it to manage.

---

Your task is to define the MLP as depicted in the figure below. Please refer to the documentation and focus on
the classes [`nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html?highlight=nn+linear#torch.nn.Linear) to define the layers and [`F.relu`](https://pytorch.org/docs/stable/generated/torch.nn.functional.relu.html?highlight=f+relu#torch.nn.functional.relu) to call the activation funtion.

<img src="img/mlp.png" width=800></img>

In [4]:
class FC(nn.Module):
    """ Standard Multi layer perceptron for classification into 10 
    classes. Consists of 4 FC layers, ReLU activations are used 
    for the first 3.
    """
    def __init__(self):
        """ Constructor, layers definitions go here. Only specify
        those layers which have any trainable parameters (but for
        instance not the activation functions as the ones we use 
        do not have any trainable parameters). """
        # The following line is needed to initialize the nn.Module properly
        super(FC, self).__init__()

        ### WRITE YOUR CODE HERE
        self.fc1 = nn.Linear(2352, 512)
        self.fc2 = nn.Linear(512, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 10)

    def forward(self, x):
        """ Feed-forward pass, this is where the actual computation happens
        and the computational graph is built (from scratch each time this 
        function is called). """
        # Note: we first flatten the images into vectors
        # This is done over the last 3 dimensions: (channel, height, width)
        x = x.flatten(-3)
        
        ### WRITE YOUR CODE HERE
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))

        return x
    
# Instantiate the model.
model_fc = FC()

**Q:** How many learnable parameters (weights) does this model have?

**A:** 784 * 512 + 512 * 50 + 50 * 50 + 50 * 10

## 1.7 Inspecting the model architecture

Let us check the model architecture and see how many trainable parameters we really use. For this purpose we will use the `torchinfo` package.

Note the number of trainable parameters.

In [6]:
summary(model_fc, input_size=(1, 3, 28, 28))  # the first dimension=1 corresponds to the batch size

Layer (type:depth-idx)                   Output Shape              Param #
FC                                       [1, 10]                   --
├─Linear: 1-1                            [1, 512]                  1,204,736
├─Linear: 1-2                            [1, 50]                   25,650
├─Linear: 1-3                            [1, 50]                   2,550
├─Linear: 1-4                            [1, 10]                   510
Total params: 1,233,446
Trainable params: 1,233,446
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 1.23
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 4.93
Estimated Total Size (MB): 4.95

## 1.8 Loss function

We are at step (3) of our pipeline. As explained above, our loss function $\mathcal{L}$ will be $\text{CE}(\hat{\mathbf{y}}, \mathbf{y})$, which is provided for us by PyTorch, please refer to the documentation of [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss).

**Note:** as explained above, this function will also directly implement the *Softmax* on the predictions, so you don't need to manually call it.

There are [many commonly used loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions) defined in the `torch.nn` module, and you can also implement your own using PyTorch operations. 

Your task is to instantiate the CE loss function.

In [7]:
# Define the loss fuction.
criterion = torch.nn.CrossEntropyLoss()

## 1.9 Optimizer
We are at step (4) of the pipeline. The [Optimizer](https://pytorch.org/docs/stable/optim.html) updates the weights of the network given their currently computed gradients. It can be a simple state-less function (such as SGD) or a more advanced one that keeps track of additional information about the weights and their gradients (such as a running mean), which can be used for more advanced update rules.

We will opt for the simplest case, the state-less Stochastic Gradient Descent. Your task is to instantiate this optimizer for the parameters (:= weights) of our model, please refer to [`torch.optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD). We will also need to define it's learning rate.

In [8]:
learning_rate = 1e-1
optimizer = torch.optim.SGD(model_fc.parameters(), lr=learning_rate)

## 1.10 Training loop

We are at step (5) of our pipeline. We would like to define a training loop where we iterate over the training samples, predict the outputs, and update the model based on its errors. Let us define a function `train_model()` that will be used for training any network architecture we come up with.

Fill in the code that follows the steps 5.2 - 5.6 of our training pipeline. For running the backward pass, use the function [`backward()`](https://pytorch.org/docs/stable/generated/torch.autograd.backward.html#torch.autograd.backward). For zeroing out the accumulated gradients, use the function [`zero_grad()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=zero_grad#torch.nn.Module.zero_grad).

In [13]:
def train_model(model, criterion, optimizer, dataloader_train, dataloader_test, epochs):
    """ Trains the model for the specified number of epochs on the dataset.

    Args:
        model: The model to train.
        criterion: The loss function.
        optimizer: The optimizer to use.
        dataloader_train: The DataLoader for the training set.
        dataloader_test: The DataLoader for the test set.
        epochs: The number of epochs to train for.
    """
    for ep in range(epochs):
        # Training.
        model.train()
        for it, batch in enumerate(dataloader_train):
            # 5.1 Load a batch, break it down in images and targets.
            x, y = batch

            # 5.2 Run forward pass.
            logits = model.forward(x)  ### WRITE YOUR CODE HERE
            
            # 5.3 Compute loss (using 'criterion').
            loss = criterion(logits, y) ### WRITE YOUR CODE HERE
            
            # 5.4 Run backward pass.
            loss.backward()
            
            # 5.5 Update the weights using 'optimizer'.
            
            # 5.6 Zero-out the accumulated gradients.
            optimizer.step()
            optimizer.zero_grad()

            print('\rEp {}/{}, it {}/{}: loss train: {:.2f}, accuracy train: {:.2f}'.
                  format(ep + 1, epochs, it + 1, len(dataloader_train), loss,
                         accuracy(logits, y)), end='')

        # Validation.
        model.eval()
        with torch.no_grad():
            acc_run = 0
            for it, batch in enumerate(dataloader_test):
                # Get batch of data.
                x, y = batch
                curr_bs = x.shape[0]
                acc_run += accuracy(model(x), y) * curr_bs
            acc = acc_run / len(dataloader_test.dataset)

            print(', accuracy test: {:.2f}'.format(acc))

### Training the model

Let's now use everything that we have prepared to train our model on MNIST:

In [12]:
epochs = 5
train_model(model_fc, criterion, optimizer, dataloader_train, dataloader_test, epochs)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

# 2 Convolutional Neural Networks (CNNs)

Our 4-layer MLP network works well, reaching a test accuracy of ~0.96. However, this network uses ~0.5M weights. We can use even deeper architectures with fewer parameters and take advantage of the 2D structure of the input data (images) using CNNs.

## 2.1 LeNet-5

Let us define a simple CNN with 2 convolutional layers with max-pooling and 3 Fully-Connected (FC) layers. In particular, we will implement a variant of the architecture called [LeNet-5 introduced by Yann LeCun in 1999](http://yann.lecun.com/exdb/publis/pdf/lecun-99.pdf). 


Your task is to define a simple LeNet-5 architecture depicted in the figure below. Print the architecture and comment on the number of parameters. Finally train the model. To specify the layers, please additionally refer to [`nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) and [`F.max_pool2d`](https://pytorch.org/docs/stable/generated/torch.nn.functional.max_pool2d.html?highlight=functional+max+pool+2d).

### A word about **padding**
As you saw in the lecture, a convolution will decrease the size of the signal if we only consider the *valid* convolutions, i.e. when the kernel (or mask) is fully within the input signal. In order to avoid this, we can allow the kernel to go slightly beyond the original signal by padding it with zeros at its beginning and end.

To visualize this, imagine a 1D signal of size $4$ and a kernel of size $3$: we can move the kernel on the input only twice, and the output will then have a size of $2$. However, if we add one $0$ at the beginning and at the end of the input, therefore changing its effective size to $6$, the kernel will be able to move four times over it, which will give an output of size $4$, the same as the original input.

**Notes for the implementation:**
* a kernel of size $k$ will lead to a decrease in the size of feature maps by $k-1$, so we should pad with zeros each side by $(k-1)/2$
* we want to use the max-pooling to reduce the size by 2, so its `kernel_size` should be 2
* to go from convolutional to fully-connected layers, we need to reshape the tensor

<img src="img/lenet5.png" width=800></img>

In [38]:
class CNN_LeNet(nn.Module):
    def __init__(self):
        super(CNN_LeNet, self).__init__()
        self.conv2d1 = nn.Conv2d(1, 6, 3)
        self.conv2d2 = nn.Conv2d(6, 16, 3)    # 13x13 → 11x11
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 11x11 → maxpool → 5x5
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv2d1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2d2(x)), 2)
        x = torch.flatten(x, 1)                       # -> (batch_size, 400)
        x = F.relu(self.fc1(x))                       # -> (batch_size, 120)
        x = F.relu(self.fc2(x))                       # -> (batch_size, 84)
        x = self.fc3(x)                               # -> (batch_size, 10)
        return x

    
# Instantiate the model.
model_lenet = CNN_LeNet()

**Q:** What is the number of trainable parameters in our LeNet model?

**A:** 

Let us check the architecture again and the number of trainable parameters. We can directly see that this architecture needs just about 20% of the parameters the MLP used.

In [39]:
model_lenet(torch.randn(1, 1, 28, 28)).shape

torch.Size([1, 10])

In [40]:
# Print out the architecture and check the number of parameters.
summary(model_lenet, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNN_LeNet                                [1, 10]                   --
├─Conv2d: 1-1                            [1, 6, 26, 26]            60
├─Conv2d: 1-2                            [1, 16, 11, 11]           880
├─Linear: 1-3                            [1, 120]                  48,120
├─Linear: 1-4                            [1, 84]                   10,164
├─Linear: 1-5                            [1, 10]                   850
Total params: 60,074
Trainable params: 60,074
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.21
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29

We can now again train our model. As the `train_model()` function we wrote is agnostic to the network used, and because PyTorch automatically computes the gradient for us, we can directly reuse it with our CNN.

However, we do need to define a new optimizer to apply SGD to the weights of our new model!

In [62]:
# Train the model
epochs = 5
learning_rate = 1e-1
optimizer_lenet = torch.optim.SGD(model_lenet.parameters(), lr=learning_rate)
train_model(model_lenet, criterion, optimizer_lenet, dataloader_train, dataloader_test, epochs)

, accuracy test: 0.89oss train: 0.55, accuracy train: 0.85
, accuracy test: 0.94oss train: 0.24, accuracy train: 0.95
, accuracy test: 0.97oss train: 0.14, accuracy train: 0.96
, accuracy test: 0.97oss train: 0.08, accuracy train: 0.98
, accuracy test: 0.97oss train: 0.07, accuracy train: 0.98


## 2.2 3-layered CNN

Let us now define an even deeper CNN with 3 convolutional layers and only 2 FC layers. This network should reach higher accuracy (or converge faster) and still use fewer parameters than the previous architectures.

Your task is to implement a 3-layer CNN as depicted in the figure below. Check the number of parameters using `torchinfo`. Train the model and play around with the number of filters (kernels) used by every layer. Comment on your findings.

<img src="img/cnn.png" width=800></img>

In [65]:
class CNN(nn.Module):
    """ CNN, expects input shape (1, 28, 28).
    """
    def __init__(self, filters=(16, 32, 64)):
        """
        Args
        ----
        filters: tuple or list of 3 integers
            The number of filters (:=kernels) used in the network.
            See the above image for reference.
        """
        super(CNN, self).__init__()

        self.conv2d1 = nn.Conv2d(1, filters[0], 3)
        self.conv2d2 = nn.Conv2d(filters[0], filters[1], 3)
        self.conv2d3 = nn.Conv2d(filters[1], filters[2], 3)

        self.fc1 = nn.Linear(filters[2] * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv2d1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2d2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2d3(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model.
filters = (16, 32, 64)
model_cnn = CNN(filters)

In [66]:
# Print out the architecture and number of parameters.
summary(model_cnn, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [1, 10]                   --
├─Conv2d: 1-1                            [1, 16, 26, 26]           160
├─Conv2d: 1-2                            [1, 32, 11, 11]           4,640
├─Conv2d: 1-3                            [1, 64, 3, 3]             18,496
├─Linear: 1-4                            [1, 120]                  7,800
├─Linear: 1-5                            [1, 10]                   1,210
Total params: 32,306
Trainable params: 32,306
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.85
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 0.13
Estimated Total Size (MB): 0.26

And again, we train our new model by reusing the same dataset, criterion, and training function, but with a new optimizer defined for this new model:

In [67]:
# Train the model.
learning_rate = 1e-1
optimizer_cnn = torch.optim.SGD(model_cnn.parameters(), lr=learning_rate) ### WRITE YOUR CODE HERE
train_model(model_cnn, criterion, optimizer_cnn, dataloader_train, dataloader_test, epochs)

, accuracy test: 0.92oss train: 0.24, accuracy train: 0.93
, accuracy test: 0.96oss train: 0.11, accuracy train: 0.95
, accuracy test: 0.96oss train: 0.17, accuracy train: 0.95
, accuracy test: 0.97oss train: 0.04, accuracy train: 0.99
, accuracy test: 0.97oss train: 0.09, accuracy train: 0.99


## 2.3 Trying out your own input

We have provided a tool for you to draw your own digits and test your network. Play around with the inputs to get a sense of how accurate your model is. Use the button `reset` to reset the canvas and `predict` to run the prediction on the current canvas image. You can use the button `blur` to blur your drawn image so that it looks closer to the samples from the training set.

**Note:** the following cell may not work properly if VS Code or jupyter lab.

In [68]:
dp = DrawingPad((28, 28), model_lenet)

<IPython.core.display.Javascript object>

Button(description='reset', style=ButtonStyle())

Button(description='blur', style=ButtonStyle())

Button(description='predict', style=ButtonStyle())

# 3 Build your own model (Optional)

In this part, we prepared some empty model definition for you. You can use it to build your own model and test it on MNIST!

In [69]:
class MyModel(nn.Module):
    """ Build your own model.
    It should take as input images of shape (1, 28, 28).
    """
    def __init__(self):
        """
        Initialize your model.
        
        Feel free to add argument if you want to.
        """
        super(MyModel, self).__init__()

        ### WRITE YOUR CODE HERE
        ...

    def forward(self, x):
        """Write the forward pass."""
        ### WRITE YOUR CODE HERE
        return ...

# Instantiate the model and print its architecture.
my_model = MyModel()
summary(my_model, input_size=(1, 1, 28, 28))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [ ]:
# Train your model.
learning_rate = 1e-1
my_optimizer = ...  ### WRITE YOUR CODE HERE
train_model(my_model, criterion, my_optimizer, dataloader_train, dataloader_test, epochs)

# 4 Written questions

**Q.1** (MCQ) Which of the following statements is/are correct?

1. Gradient Descent updates the weights of the network in the direction opposite to the gradient of the loss function.
2. The primary role of an activation function is to introduce non-linearity into the deep network.
3. Strided convolution (stride > 1) is used to increase the size of the feature maps.
4. Max pooling has learnable parameters.

**A.1** 

**Q.2** What advantages do CNN have over MLP? Can we always use a CNN instead of an MLP?

**A.2** 

**Q.3** Consider the following 1D signal:

$$[-1, 4, -6, 0, 3, 2, 1, -3, 5]$$

What is the output of the 1D convolution with the mask $w$ and $\texttt{stride=2}$ (no padding is used)?

$$w = [-1, 2, 1]$$

**A.3** 

**Q.4** Imagine you have a CNN that you use to classify greyscale images of animals, where each image is of size $128\times128\times1$. You then receive an updated version of that dataset where the images are in color, so they have now three channel: Red, Green, and Blue.

What is the new shape of the images? What minial change to your CNN can you do to make it accept and use this new color information?

**A.4** 